In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import matplotlib, os, datetime, sys, os
from matplotlib import style
from scipy import stats
style.use('seaborn')
sns.set(font_scale=1.4)

import geopandas as gp
import sqlalchemy as sql
import json
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Clear this before publishing
creds = {'un': 'SANDAGNET\\nmad', 'pw': '}
engine = sql.create_engine('mssql+pymssql://%s:%s@DDAMWSQL16/demographic_warehouse' % (creds['un'], creds['pw']))
engine2 = sql.create_engine('mssql+pymssql://%s:%s@sql2014b8/GeoDepot' % (creds['un'], creds['pw']))

In [4]:
# Reading edd_sector data
query = '''
SELECT *
  FROM [socioec_data].ca_edd.[xref_sandag_industry_edd_sector]
'''
xref_edd_sandag = pd.read_sql(query, con=engine.connect())
xref_edd_sandag['edd_sector_v2'] = xref_edd_sandag['edd_sector'].str.replace('-', '').astype('int')
print("xref_edd_sandag shape: ", xref_edd_sandag.shape)
xref_edd_sandag.head()

xref_edd_sandag shape:  (26, 5)


,yr,sandag_industry_id,edd_sector,sandag_sector,edd_sector_v2
0,2013,1,11-000000,11,11000000
1,2013,2,10-000000,21,10000000
2,2013,3,43-220000,22,43220000
3,2013,4,20-000000,23,20000000
4,2013,5,30-000000,31-33,30000000


In [7]:
# Read CES data which was downloaded for SD county from the site https://data.edd.ca.gov/Industry-Information-/Current-Employment-Statistics-CES-Annual-Average-1/pwx8-ztk5/data
ces_df = pd.read_csv('Current_Employment_Statistics__CES___Annual_Average__1990_-_2021 (1).csv', low_memory=False)
ces_df = ces_df[ces_df['Year'] == 2019].sort_values(by = 'Series Code')
print('ind_quarter_df shape: ', ces_df.shape)
ces_df.head()

ind_quarter_df shape:  (100, 8)


,Area Type,Area Name,Year,Period,Series Code,Industry Title,Seasonally Adjusted (Y/N),Current Employment
2945,County,San Diego County,2019,Annual,0,Total Nonfarm,N,1503200
2946,County,San Diego County,2019,Annual,1000000,Total Wage and Salary,N,1512900
2947,County,San Diego County,2019,Annual,5000000,Total Private,N,1254600
2948,County,San Diego County,2019,Annual,6000000,Goods Producing,N,200000
2949,County,San Diego County,2019,Annual,7000000,Service Providing,N,1303200


In [9]:
ces_df['agg_code'] = ces_df['Series Code'] // 10**6
ces_df['agg_code'].head(15)

2945     0
2946     1
2947     5
2948     6
2949     7
2950     8
2951    10
2952    11
2953    15
2954    20
2955    20
2956    20
2957    20
2958    20
2959    20
Name: agg_code, dtype: int64

In [6]:
ces_df.dtypes

Area Type                    object
Area Name                    object
Year                          int64
Period                       object
Series Code                   int64
Industry Title               object
Seasonally Adjusted (Y/N)    object
Current Employment            int64
dtype: object

__Mapping industry categories from edd to sandag__

In [10]:
temp = pd.merge(ces_df, xref_edd_sandag[['sandag_industry_id', 'edd_sector_v2']],
               left_on = 'Series Code', right_on = 'edd_sector_v2', how = 'left')
temp.head()

,Area Type,Area Name,Year,Period,Series Code,Industry Title,Seasonally Adjusted (Y/N),Current Employment,agg_code,sandag_industry_id,edd_sector_v2
0,County,San Diego County,2019,Annual,0,Total Nonfarm,N,1503200,0,NaN,NaN
1,County,San Diego County,2019,Annual,1000000,Total Wage and Salary,N,1512900,1,NaN,NaN
2,County,San Diego County,2019,Annual,5000000,Total Private,N,1254600,5,NaN,NaN
3,County,San Diego County,2019,Annual,6000000,Goods Producing,N,200000,6,NaN,NaN
4,County,San Diego County,2019,Annual,7000000,Service Providing,N,1303200,7,NaN,NaN


In [19]:
# final_df is created by removing the na sub categories
final_df = temp[~temp['sandag_industry_id'].isna()]

In [20]:
final_df

,Area Type,Area Name,Year,Period,Series Code,Industry Title,Seasonally Adjusted (Y/N),Current Employment,agg_code,sandag_industry_id,edd_sector_v2
6,County,San Diego County,2019,Annual,10000000,Mining and Logging,N,400,10,2.0,10000000.0
7,County,San Diego County,2019,Annual,11000000,Total Farm,N,9700,11,1.0,11000000.0
9,County,San Diego County,2019,Annual,20000000,Construction,N,84000,20,4.0,20000000.0
16,County,San Diego County,2019,Annual,30000000,Manufacturing,N,115700,30,5.0,30000000.0
24,County,San Diego County,2019,Annual,41000000,Wholesale Trade,N,44000,41,6.0,41000000.0
27,County,San Diego County,2019,Annual,42000000,Retail Trade,N,145800,42,7.0,42000000.0
41,County,San Diego County,2019,Annual,43220000,Utilities,N,4600,43,3.0,43220000.0
42,County,San Diego County,2019,Annual,43400089,Transportation and Warehousing,N,29700,43,8.0,43400089.0
44,County,San Diego County,2019,Annual,50000000,Information,N,23500,50,9.0,50000000.0
50,County,San Diego County,2019,Annual,55520000,Finance and Insurance,N,46400,55,10.0,55520000.0


In [21]:
final_df.to_csv('2019_emp_total_merged.csv')